# Part 2: Python Integration

## Database Connection and Connection Test

In [1]:
pip install psycopg2

Note: you may need to restart the kernel to use updated packages.


In [2]:
import psycopg2
from psycopg2 import sql, extras

# Database connection parameters
db_config = {
    'dbname': 'Huyen Truong 2',
    'user': 'postgres',
    'password': 'Huyen30032004@',
    'host': 'localhost',
    'port': 5432
}

In [3]:
# Function to connect to the database
def connect_db():
    try:
        conn = psycopg2.connect(**db_config)
        conn.autocommit = True
        return conn
    except Exception as e:
        print("Error connecting to the database:", e)
        return None

In [4]:
# Test connection with a simple query
def test_connection():
    conn = connect_db()
    if conn is not None:
        try:
            # Create a cursor
            cursor = conn.cursor()
            
            # Execute a simple query
            cursor.execute("SELECT CURRENT_DATE;")
            current_date = cursor.fetchone()
            print("Connection test successful. Current date:", current_date[0])

            # Close cursor and connection
            cursor.close()
            conn.close()
        except Exception as e:
            print("Error executing the test query:", e)
    else:
        print("Connection test failed.")

# Run the test
test_connection()

Connection test successful. Current date: 2024-11-08


# 1. Calling the Stored Procedure

In [5]:
# 1. Calling the Stored Procedure for Enrollment
def enroll_student(student_id, course_id):
    conn = connect_db()
    if not conn:
        print("Unable to establish a database connection.")
        return

    try:
        with conn.cursor() as cursor:
            # Use the CALL statement to invoke the stored procedure
            cursor.execute(f"CALL EnrollStudent({student_id}, {course_id});")
            print(f"Enrollment successful for Student ID {student_id} in Course ID {course_id}")

    except psycopg2.Error as e:
        # Check specific error messages
        if 'course is full' in str(e).lower():
            print(f"Enrollment failed: Course ID {course_id} is full.")
        elif 'already enrolled' in str(e).lower():
            print(f"Enrollment failed: Student ID {student_id} is already enrolled in Course ID {course_id}.")
        else:
            print(f"Enrollment failed for Student ID {student_id} in Course ID {course_id}:", e)
    
    finally:
        conn.close()

# 2. Querying the Views

In [6]:
pip install tabulate

Note: you may need to restart the kernel to use updated packages.


In [7]:
# 2. Querying the Views

from tabulate import tabulate

# Function to query the StudentCourseView
def query_student_course_view():
    conn = connect_db()
    if not conn:
        return

    try:
        with conn.cursor(cursor_factory=extras.DictCursor) as cursor:
            cursor.execute("SELECT * FROM StudentCourseView")
            records = cursor.fetchall()
            # Convert records to a list of lists for tabulate
            table_data = [dict(record) for record in records]
            print("Student Course View Results:")
            print(tabulate(table_data, headers="keys", tablefmt="grid"))
    except Exception as e:
        print("Error querying StudentCourseView:", e)
    finally:
        conn.close()

# Function to query the CourseCapacityView
def query_course_capacity_view():
    conn = connect_db()
    if not conn:
        return

    try:
        with conn.cursor(cursor_factory=extras.DictCursor) as cursor:
            cursor.execute("SELECT * FROM CourseCapacityView")
            records = cursor.fetchall()
            # Convert records to a list of lists for tabulate
            table_data = [dict(record) for record in records]
            print("Course Capacity View Results:")
            print(tabulate(table_data, headers="keys", tablefmt="grid"))
    except Exception as e:
        print("Error querying CourseCapacityView:", e)
    finally:
        conn.close()


# 3. Triggering Events

In [8]:
# 3. Triggering Events by Insert and Delete Operations
def test_trigger_insert(student_id, course_id):
    conn = connect_db()
    if not conn:
        return

    try:
        with conn.cursor() as cursor:
            cursor.execute("INSERT INTO Enrollments (student_id, course_id) VALUES (%s, %s)", (student_id, course_id))
            print(f"Added enrollment for Student ID {student_id} in Course ID {course_id}")
    except Exception as e:
        print("Error triggering insert:", e)
    finally:
        conn.close()

def test_trigger_delete(enrollment_id):
    conn = connect_db()
    if not conn:
        return

    try:
        with conn.cursor() as cursor:
            cursor.execute("DELETE FROM Enrollments WHERE enrollment_id = %s", (enrollment_id,))
            print(f"Removed enrollment with Enrollment ID {enrollment_id}")
    except Exception as e:
        print("Error triggering delete:", e)
    finally:
        conn.close()

# Test Plan: University Enrollment System

### Review the "Students" and "Courses" Tables

Before proceeding with the rest of the operations, let's examine the contents of the "Students" and "Courses" tables to ensure that we have the necessary data. The following code will query both tables and display their contents in a well-structured tabular format.

- **Students Table**: Contains information about the students enrolled in various courses.
- **Courses Table**: Lists the courses available for students to enroll in.

The tables are displayed using the `tabulate` library for better readability.


In [9]:
from tabulate import tabulate

# Function to query the Students table and display it in tabular format
def query_students_table():
    conn = connect_db()
    if not conn:
        return

    try:
        with conn.cursor(cursor_factory=extras.DictCursor) as cursor:
            cursor.execute("SELECT * FROM Students;")
            records = cursor.fetchall()
            
            # Convert records to a list of dictionaries for tabulate
            table_data = [dict(record) for record in records]
            if table_data:
                print("Students Table contains the following records:")
                print(tabulate(table_data, headers="keys", tablefmt="grid"))
            else:
                print("No records found in the Students table.")
    
    except Exception as e:
        print("Error querying Students table:", e)
    finally:
        conn.close()

# Function to query the Courses table and display it in tabular format
def query_courses_table():
    conn = connect_db()
    if not conn:
        return

    try:
        with conn.cursor(cursor_factory=extras.DictCursor) as cursor:
            cursor.execute("SELECT * FROM Courses;")
            records = cursor.fetchall()
            
            # Convert records to a list of dictionaries for tabulate
            table_data = [dict(record) for record in records]
            if table_data:
                print("Courses Table contains the following records:")
                print(tabulate(table_data, headers="keys", tablefmt="grid"))
            else:
                print("No records found in the Courses table.")
    
    except Exception as e:
        print("Error querying Courses table:", e)
    finally:
        conn.close()

# Query and display the Students and Courses tables
query_students_table()
query_courses_table()


Students Table contains the following records:
+--------------+--------------+-------------+-------------------------+
|   student_id | first_name   | last_name   | major                   |
+==============+==============+=============+=========================+
|            1 | Ngoc Huyen   | Truong      | Data Science            |
+--------------+--------------+-------------+-------------------------+
|            2 | John         | Doe         | Computer Science        |
+--------------+--------------+-------------+-------------------------+
|            3 | Jane         | Smith       | Mechanical Engineering  |
+--------------+--------------+-------------+-------------------------+
|            4 | Alice        | Brown       | Electrical Engineering  |
+--------------+--------------+-------------+-------------------------+
|            5 | Robert       | Johnson     | Business Administration |
+--------------+--------------+-------------+-------------------------+
Courses Table con

Before proceeding with the test cases, I created two functions:
- **Function to query the Enrollments table**: This function checks the Enrollments table to verify if a student has been successfully enrolled in a course.
- **Function to query the EnrollmentLog table**: This function queries the EnrollmentLog table to ensure that an enrollment record was added when the student enrolls or drops a course.

In [10]:
# Function to query the Enrollments table
def query_enrollments_table():
    conn = connect_db()
    if not conn:
        return

    try:
        with conn.cursor(cursor_factory=extras.DictCursor) as cursor:
            # Query to get all records from the Enrollments table
            cursor.execute("SELECT * FROM Enrollments")
            records = cursor.fetchall()
            
            # Convert records to a list of dictionaries for tabulate
            table_data = [dict(record) for record in records]
            if table_data:
                print("Enrollments Table contains the following records:")
                print(tabulate(table_data, headers="keys", tablefmt="grid"))
            else:
                print("No records found in the Enrollments table.")
    
    except Exception as e:
        print("Error querying Enrollments table:", e)
    finally:
        conn.close()


In [11]:
# Function to query the EnrollmentLog table
def query_enrollment_log():
    conn = connect_db()  
    if not conn:
        return

    try:
        with conn.cursor(cursor_factory=extras.DictCursor) as cursor:
            # Query to get all records from the EnrollmentLog table
            cursor.execute("SELECT * FROM EnrollmentLog")
            records = cursor.fetchall()
            
            # Convert records to a list of dictionaries for tabulate
            table_data = [dict(record) for record in records]
            
            if table_data:
                print("EnrollmentLog Table contains the following logs:")
                print(tabulate(table_data, headers="keys", tablefmt="grid"))
            else:
                print("No records found in the EnrollmentLog table.")
    
    except Exception as e:
        print("Error querying EnrollmentLog table:", e)
    finally:
        conn.close()


# Test Case 0: Successful Enrollment

In [12]:
# Test Case 0: Successful Enrollment
def test_case_successful_enrollment():
    student_id = 1  # Ngoc Huyen Truong
    course_id = 1   # Data Structures

    # Step 1: Call the enrollment function to enroll the student
    print("Running Test Case 0: Successful Enrollment")
    enroll_student(student_id, course_id)
    
    # Step 2: Query the Enrollments table to check if the record has been inserted
    print("\n--- Enrollments Table after Enrollment ---")
    query_enrollments_table()
    
    # Step 3: Query the StudentCourseView to confirm the enrollment
    print("\n--- StudentCourseView after Enrollment ---")
    query_student_course_view()
    
    # Step 4: Query the CourseCapacityView to check the updated capacity
    print("\n--- CourseCapacityView after Enrollment ---")
    query_course_capacity_view()
    
    # Step 5: Query the EnrollmentLog table to ensure the enrollment event was logged
    print("\n--- EnrollmentLog Table after Enrollment ---")
    query_enrollment_log()
    

In [13]:
# Run Test Case 0
test_case_successful_enrollment()

Running Test Case 0: Successful Enrollment
Enrollment successful for Student ID 1 in Course ID 1

--- Enrollments Table after Enrollment ---
Enrollments Table contains the following records:
+-----------------+--------------+-------------+----------------------------+
|   enrollment_id |   student_id |   course_id | enrollment_date            |
+=================+==============+=============+============================+
|               1 |            1 |           1 | 2024-11-08 16:46:38.309118 |
+-----------------+--------------+-------------+----------------------------+

--- StudentCourseView after Enrollment ---
Student Course View Results:
+--------------+-------------------+-----------------+----------------------------+
|   student_id | student_name      | course_name     | enrollment_date            |
+==============+===================+=================+============================+
|            1 | Ngoc Huyen Truong | Data Structures | 2024-11-08 16:46:38.309118 |
+---------

# Test Case 1: Course Capacity Update After Multiple Enrollments

In [14]:
# Test Case 1: Course Capacity Update After Multiple Enrollments

def test_case_multiple_enrollments():
    # Initial data for test
    course_id = 4  #"Marketing Principles" with max_capacity = 6
    student_ids = [1, 2, 3]  # List of student IDs to enroll

    # Enroll each student in the course one by one
    for student_id in student_ids:
        print(f"\nEnrolling Student ID {student_id} in Course ID {course_id}")
        
        # Enroll student
        enroll_student(student_id, course_id)
        
        # Query the CourseCapacityView to verify capacity update
        print("Course Capacity View after enrollment:")
        query_course_capacity_view()
        
    # Query the Enrollments table to verify enrollment record was added
    print("Enrollments Table after enrollment:")
    query_enrollments_table()

# Run Test Case 1
test_case_multiple_enrollments()



Enrolling Student ID 1 in Course ID 4
Enrollment successful for Student ID 1 in Course ID 4
Course Capacity View after enrollment:
Course Capacity View Results:
+-------------+----------------------+-------------------------+----------------------+----------------------+
|   course_id | course_name          | department              |   current_enrollment |   remaining_capacity |
+=============+======================+=========================+======================+======================+
|           4 | Marketing Principles | Business Administration |                    1 |                    5 |
+-------------+----------------------+-------------------------+----------------------+----------------------+
|           2 | Linear Algebra       | Mathematics             |                    0 |                    4 |
+-------------+----------------------+-------------------------+----------------------+----------------------+
|           1 | Data Structures      | Computer Science      

# Test Case 2: Event Logging for Enrollment

In [15]:
# Test Case 2: Event Logging for Enrollment

def test_case_event_logging():
    # Initial data for test
    student_id = 4  # Student ID 4 (Alice Brown) exists
    course_id = 3   # "Physics" course with available capacity

    # Step 1: Enroll the student in the course
    print(f"\nEnrolling Student ID {student_id} in Course ID {course_id}")
    enroll_student(student_id, course_id)
    
    # Step 2: Query the EnrollmentLog table to verify action 'ENROLL' was logged
    print("\nChecking EnrollmentLog for enrollment action:")
    query_enrollment_log()

    # Step 3: Query the StudentCourseView to confirm the enrollment
    print("\nChecking StudentCourseView for confirmation of enrollment:")
    query_student_course_view()

    # Step 4: Query the CourseCapacityView to ensure capacity was reduced by 1
    print("\nChecking CourseCapacityView for updated remaining capacity:")
    query_course_capacity_view()

# Run Test Case 2
test_case_event_logging()



Enrolling Student ID 4 in Course ID 3
Enrollment successful for Student ID 4 in Course ID 3

Checking EnrollmentLog for enrollment action:
EnrollmentLog Table contains the following logs:
+----------+----------+--------------+-------------+----------------------------+
|   log_id | action   |   student_id |   course_id | event_time                 |
+==========+==========+==============+=============+============================+
|        1 | ENROLL   |            1 |           1 | 2024-11-08 16:46:38.309118 |
+----------+----------+--------------+-------------+----------------------------+
|        2 | ENROLL   |            1 |           4 | 2024-11-08 16:46:46.629824 |
+----------+----------+--------------+-------------+----------------------------+
|        3 | ENROLL   |            2 |           4 | 2024-11-08 16:46:46.757886 |
+----------+----------+--------------+-------------+----------------------------+
|        4 | ENROLL   |            3 |           4 | 2024-11-08 16:46:46.

# Test Case 3: Double Enrollment Prevention

In [16]:
#Query to confirm precondition: Student id 4 is already enrolled in course_id = 3
query_enrollments_table()

Enrollments Table contains the following records:
+-----------------+--------------+-------------+----------------------------+
|   enrollment_id |   student_id |   course_id | enrollment_date            |
+=================+==============+=============+============================+
|               1 |            1 |           1 | 2024-11-08 16:46:38.309118 |
+-----------------+--------------+-------------+----------------------------+
|               2 |            1 |           4 | 2024-11-08 16:46:46.629824 |
+-----------------+--------------+-------------+----------------------------+
|               3 |            2 |           4 | 2024-11-08 16:46:46.757886 |
+-----------------+--------------+-------------+----------------------------+
|               4 |            3 |           4 | 2024-11-08 16:46:46.886896 |
+-----------------+--------------+-------------+----------------------------+
|               5 |            4 |           3 | 2024-11-08 16:46:51.214548 |
+-------------

In [17]:
# Test Case 3: Double Enrollment Prevention

def test_case_double_enrollment_prevention():
    # Initial data for test
    student_id = 4  # Student ID 4 (Alice Brown) exists
    course_id = 3   # "Physics" 

    # Step 1: Attempt to enroll the student in the same course again
    enroll_student(student_id, course_id) 
    
    # Step 2: Query the Enrollments table to confirm no duplicate enrollment
    print("\nChecking Enrollments table to ensure no duplicate enrollment exists:")
    query_enrollments_table() 

    # Step 3: Verify that the EnrollmentLog table has no added records
    print("\nChecking EnrollmentLog for no added action:")
    query_enrollment_log()  

# Run Test Case 3
test_case_double_enrollment_prevention()


Enrollment failed: Student ID 4 is already enrolled in Course ID 3.

Checking Enrollments table to ensure no duplicate enrollment exists:
Enrollments Table contains the following records:
+-----------------+--------------+-------------+----------------------------+
|   enrollment_id |   student_id |   course_id | enrollment_date            |
+=================+==============+=============+============================+
|               1 |            1 |           1 | 2024-11-08 16:46:38.309118 |
+-----------------+--------------+-------------+----------------------------+
|               2 |            1 |           4 | 2024-11-08 16:46:46.629824 |
+-----------------+--------------+-------------+----------------------------+
|               3 |            2 |           4 | 2024-11-08 16:46:46.757886 |
+-----------------+--------------+-------------+----------------------------+
|               4 |            3 |           4 | 2024-11-08 16:46:46.886896 |
+-----------------+-------------

# Test Case 4: Successful Unenrollment

In [18]:
# Initial check to confirm that student_id = 4 is enrolled in course_id = 3
print("Precondition check: Confirming initial enrollment exists for unenrollment.")
query_enrollments_table()

Precondition check: Confirming initial enrollment exists for unenrollment.
Enrollments Table contains the following records:
+-----------------+--------------+-------------+----------------------------+
|   enrollment_id |   student_id |   course_id | enrollment_date            |
+=================+==============+=============+============================+
|               1 |            1 |           1 | 2024-11-08 16:46:38.309118 |
+-----------------+--------------+-------------+----------------------------+
|               2 |            1 |           4 | 2024-11-08 16:46:46.629824 |
+-----------------+--------------+-------------+----------------------------+
|               3 |            2 |           4 | 2024-11-08 16:46:46.757886 |
+-----------------+--------------+-------------+----------------------------+
|               4 |            3 |           4 | 2024-11-08 16:46:46.886896 |
+-----------------+--------------+-------------+----------------------------+
|               5

In [19]:
# Test Case 4: Successful Unenrollment

def test_case_successful_unenrollment():
    # Initial data for the test
    enrollment_id = 5
    student_id = 4 # Student ID 4 (Alice Brown) exists and is enrolled in Course ID 3
    course_id = 3   #Physics

    # Step 1: Remove the enrollment for student_id = 4 in course_id = 3 with enrollment_id = 5
    print(f"\nAttempting to unenroll Student ID {student_id} from Course ID {course_id}.")
    test_trigger_delete(enrollment_id)  

    # Step 2: Query the CourseCapacityView to ensure the remaining capacity increased by 1
    print("\nChecking CourseCapacityView to confirm capacity has increased:")
    query_course_capacity_view()  

    # Step 3: Verify that the EnrollmentLog records the DROP action
    print("\nChecking EnrollmentLog for the 'DROP' action:")
    query_enrollment_log()  

    # Step 4: Final check to confirm unenrollment in Enrollments table
    print("\nFinal confirmation in Enrollments table to ensure no record exists for this unenrollment:")
    query_enrollments_table()  # Should not have any record for student_id = 1 in course_id = 1

# Run Test Case 4
test_case_successful_unenrollment()



Attempting to unenroll Student ID 4 from Course ID 3.
Removed enrollment with Enrollment ID 5

Checking CourseCapacityView to confirm capacity has increased:
Course Capacity View Results:
+-------------+----------------------+-------------------------+----------------------+----------------------+
|   course_id | course_name          | department              |   current_enrollment |   remaining_capacity |
+=============+======================+=========================+======================+======================+
|           4 | Marketing Principles | Business Administration |                    3 |                    3 |
+-------------+----------------------+-------------------------+----------------------+----------------------+
|           2 | Linear Algebra       | Mathematics             |                    0 |                    4 |
+-------------+----------------------+-------------------------+----------------------+----------------------+
|           1 | Data Structures   

# Test Case 5: Full Course Enrollment

In [20]:
# Data Preparation: Enroll students to fill the course capacity
course_id = 2  # "Linear Algebra" with max_capacity = 4
student_ids = [1, 2, 3, 4]  # List of student IDs to enroll

# Enroll each student in the course one by one
for student_id in student_ids:
    print(f"\nEnrolling Student ID {student_id} in Course ID {course_id}")
    enroll_student(student_id, course_id)

# Precondition Check: Confirm course capacity and enrollment status
print("\nPrecondition check: Confirming course capacity and current enrollments before testing full course enrollment.")
query_course_capacity_view()  # Verify that the course is at full capacity
query_enrollments_table()      # Confirm the enrolled students



Enrolling Student ID 1 in Course ID 2
Enrollment successful for Student ID 1 in Course ID 2

Enrolling Student ID 2 in Course ID 2
Enrollment successful for Student ID 2 in Course ID 2

Enrolling Student ID 3 in Course ID 2
Enrollment successful for Student ID 3 in Course ID 2

Enrolling Student ID 4 in Course ID 2
Enrollment successful for Student ID 4 in Course ID 2

Precondition check: Confirming course capacity and current enrollments before testing full course enrollment.
Course Capacity View Results:
+-------------+----------------------+-------------------------+----------------------+----------------------+
|   course_id | course_name          | department              |   current_enrollment |   remaining_capacity |
+=============+======================+=========================+======================+======================+
|           4 | Marketing Principles | Business Administration |                    3 |                    3 |
+-------------+----------------------+-----

In [21]:
# Test case5: Full Course Enrollment
def test_case_full_course_enrollment():
    # Initial data for test
    student_id = 5  # Student ID 5 (Robert Johnson) exists
    course_id = 2   # Course ID 2 exists and has max_capacity = 4

    # Step 1: Attempt to enroll the student in a full course
    print(f"\nAttempting to enroll Student ID {student_id} in Course ID {course_id} (which is already full).")
    enroll_student(student_id, course_id)  # This should trigger the "course is full" check

    # Step 2: Query the Enrollments table to check if the enrollment failed
    print("\nChecking Enrollments table to confirm no enrollment was added for this full course:")
    query_enrollments_table()  # Should not show a new entry for student_id = 5 in course_id = 2

    # Step 3: Verify no change in the CourseCapacityView
    print("\nChecking CourseCapacityView to ensure the remaining capacity is still 0 (indicating course is full):")
    query_course_capacity_view()  # Should still indicate the course is at full capacity

# Run Test Case 5
test_case_full_course_enrollment()



Attempting to enroll Student ID 5 in Course ID 2 (which is already full).
Enrollment failed for Student ID 5 in Course ID 2: Enrollment failed: Course at full capacity
CONTEXT:  PL/pgSQL function enrollstudent(integer,integer) line 27 at RAISE


Checking Enrollments table to confirm no enrollment was added for this full course:
Enrollments Table contains the following records:
+-----------------+--------------+-------------+----------------------------+
|   enrollment_id |   student_id |   course_id | enrollment_date            |
+=================+==============+=============+============================+
|               1 |            1 |           1 | 2024-11-08 16:46:38.309118 |
+-----------------+--------------+-------------+----------------------------+
|               2 |            1 |           4 | 2024-11-08 16:46:46.629824 |
+-----------------+--------------+-------------+----------------------------+
|               3 |            2 |           4 | 2024-11-08 16:46:46.7578

# Test case 6: Prevent Enrollment If Student ID Does Not Exist

In [22]:
#Test case 6: Prevent Enrollment If Student ID Does Not Exist
    
def test_case_non_existent_student_enrollment():
    student_id = 999  # Non-existent student ID
    course_id = 4     # Course ID = 4 (Marketing Principles) exists in Courses table
        
    # Step 1: Attempt to enroll the non-existent student
    enroll_student(student_id, course_id)
    
    # Step 2: Query Enrollments table to confirm no new entry
    query_enrollments_table()

# Run Test Case 6
test_case_non_existent_student_enrollment()

Enrollment failed for Student ID 999 in Course ID 4: Enrollment failed: Student ID 999 does not exist
CONTEXT:  PL/pgSQL function enrollstudent(integer,integer) line 8 at RAISE

Enrollments Table contains the following records:
+-----------------+--------------+-------------+----------------------------+
|   enrollment_id |   student_id |   course_id | enrollment_date            |
+=================+==============+=============+============================+
|               1 |            1 |           1 | 2024-11-08 16:46:38.309118 |
+-----------------+--------------+-------------+----------------------------+
|               2 |            1 |           4 | 2024-11-08 16:46:46.629824 |
+-----------------+--------------+-------------+----------------------------+
|               3 |            2 |           4 | 2024-11-08 16:46:46.757886 |
+-----------------+--------------+-------------+----------------------------+
|               4 |            3 |           4 | 2024-11-08 16:46:46.8

# Test case 7: Attempt to Enroll Student in a Non-Existent Course

In [23]:
# Test case 7: Attempt to Enroll Student in a Non-Existent Course

def test_case_non_existent_course_enrollment():
    student_id = 2  # Student ID 2 (John Doe) exists in Students table
    course_id = 10  # Non-existent course ID

    # Step 1: Attempt to enroll the student in a non-existent course
    enroll_student(student_id, course_id)
    
    # Step 2: Query Enrollments table to confirm no new entry
    query_enrollments_table()

# Run Test Case 7
test_case_non_existent_course_enrollment()

Enrollment failed for Student ID 2 in Course ID 10: Enrollment failed: Course ID 10 does not exist
CONTEXT:  PL/pgSQL function enrollstudent(integer,integer) line 13 at RAISE

Enrollments Table contains the following records:
+-----------------+--------------+-------------+----------------------------+
|   enrollment_id |   student_id |   course_id | enrollment_date            |
+=================+==============+=============+============================+
|               1 |            1 |           1 | 2024-11-08 16:46:38.309118 |
+-----------------+--------------+-------------+----------------------------+
|               2 |            1 |           4 | 2024-11-08 16:46:46.629824 |
+-----------------+--------------+-------------+----------------------------+
|               3 |            2 |           4 | 2024-11-08 16:46:46.757886 |
+-----------------+--------------+-------------+----------------------------+
|               4 |            3 |           4 | 2024-11-08 16:46:46.886